## Контрастное обучение с SimCLR
[SimCLR](https://arxiv.org/abs/2002.05709) (Simple Contrastive Learning Representation): self-supervised модель, которая используется для получения осмысленных представлений изображений

<!-- <img src="../images/simclr_im1.png" alt="drawing" width="600"/> -->



## Contrastive learning framework

<!-- <img src="../images/simclr_im2.png" alt="drawing" width="700"/> -->

## Обучение на CIFAR-10
Обучим модель извлечения признаков изображений на наборе данных [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html). Для обучения эмбеддингов будем использовать контрастную функцию потерь.

Из набора данных выбираем $N$ изображений и для каждого из них получаем 2 избражения, используя 2 случаных преобразования (кроп, изменение цвета) исходного изображения. Пару изображений, полученных от одного изображения будем называть положительной парой, иначе отрицательной. Теперь мы имеем $2N$ изображений

Для каждой пары положительной пары $(i,j)$ определим функцию потерь, которая вынуждает модель выдавать близкие по метрике эмбеддинги для положительных пар, и далёкие для отрицательных.

$$
l_{i,j} = -\log\frac{\exp(\text{sim}(\textbf{z}_i, \textbf{z}_j)/\tau)}{\sum_{k=1}^{2N}\mathbb{1}_{[k\neq i]}\exp(\text{sim}(\textbf{z}_i,\textbf{z}_k)/\tau)}
$$

$$
\text{sim}(\textbf{u}, \textbf{v}) = \textbf{u}^T\textbf{v}/\left\lVert\textbf{u}\right\rVert \left\lVert\textbf{v}\right\rVert
$$

Итоговая функция потерь:

$$
\mathcal{L} = \frac{1}{2N}\sum_{k=1}^N[l(2k-1,2k) + l(2k, 2k - 1)]
$$


In [10]:
import torchvision
import torch
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
from torchvision import transforms

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Задание модели

В качестве бекбона будем использовать модификацию [ResNet-50](https://arxiv.org/abs/1512.03385)

Так как разрешение изображений в наборе данных CIFAR-10, меньше чем в на наборе данных [ImageNet](https://www.image-net.org/), мы заменим первый свёрточный слой с ядром $(7\times 7)$ и страйдом $2$, на свёрточный слой с ядром размера $(3\times3)$ и страйдом $1$  
Также мы удалим первый maxpolling слой

In [11]:
def get_color_distortion(s=0.5):
    # s is the strength of color distortion.
    color_jitter = transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
    rnd_color_jitter = transforms.RandomApply([color_jitter], p=0.8)
    rnd_gray = transforms.RandomGrayscale(p=0.2)
    color_distort = transforms.Compose([rnd_color_jitter, rnd_gray])
    return color_distort

In [39]:
resnet50 = torchvision.models.resnet50(pretrained=False).to(device)
list(resnet50.children())[:4]

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)]

In [34]:
new_modules = [
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1)),
    nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
] + list(resnet50.children())[4:-1]
resnet50_cifar = torch.nn.Sequential(*new_modules).to(device)

# summary(resnet50_cifar, (3, 32, 32))

In [98]:
class SimCLR(nn.Module):
    def __init__(
        self,
        base_encoder: torch.nn.Module,
        projection_head_hidden_dim=512,
        output_dim=128,
        temp=1,
    ) -> None:
        super().__init__()
        # define transforms
        self.transformations = transforms.Compose(
            [
                transforms.RandomResizedCrop(size=(32, 32)),
                transforms.RandomHorizontalFlip(),
                get_color_distortion(s=0.5),
            ]
        )

        # define base_encoder x -> h
        self.base_encoder = base_encoder

        # define projection head h -> z
        self.projection_head = nn.Sequential(
            nn.Linear(2048, projection_head_hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(projection_head_hidden_dim, output_dim),
        )

        self.temp = temp

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        Takes batch of unprocessed images, and computes contrastive loss
        """

        # # compute 2 views of each input image
        first_view = self.transformations(X)
        second_view = self.transformations(X)

        # first_view = []
        # second_view = []
        # for image in X:
        #     first_view.append(torch.unsqueeze(self.transformations(image), dim=0))
        #     second_view.append(torch.unsqueeze(self.transformations(image), dim=0))

        # compute embeddings

        first_view_emb = self.projection_head(
            torch.squeeze(self.base_encoder(first_view))
        )
        first_view_emb = first_view_emb / torch.functional.norm(
            first_view_emb, dim=1, keepdim=True
        )

        second_view_emb = self.projection_head(
            torch.squeeze(self.base_encoder(second_view))
        )
        second_view_emb = second_view_emb / torch.functional.norm(
            second_view_emb, dim=1, keepdim=True
        )

        # compute similarities
        view_cat = torch.cat([first_view_emb, second_view_emb], dim=0)  # 2N x d
        s = view_cat @ view_cat.T  # 2N x 2N

        s = torch.exp(s / self.temp)

        # compute loss
        s = s.fill_diagonal_(0)

        batch_size = X.shape[0]

        positive_pairs = torch.cat(
            [torch.diagonal(s, batch_size), torch.diagonal(s, -batch_size)], dim=0
        )

        loss = torch.mean(-torch.log(positive_pairs / (torch.sum(s, dim=1))))

        return loss

### 1) Загрузка данных

In [99]:
DATA_PATH = "../data"
BATCH_SIZE = 16
NUM_WORKERS = 4

transform = transforms.ToTensor()

cifar10_train = torchvision.datasets.CIFAR10(
    root=DATA_PATH, download=True, transform=transform, train=True
)
trainloader = torch.utils.data.DataLoader(
    cifar10_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
)

cifar10_test = torchvision.datasets.CIFAR10(
    root=DATA_PATH, download=True, transform=transform, train=False
)

trainloader = torch.utils.data.DataLoader(
    cifar10_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
)

Files already downloaded and verified
Files already downloaded and verified


In [100]:
model = SimCLR(base_encoder=resnet50_cifar).to(device)

In [101]:
X = next(iter(trainloader))[0].to(device)

In [102]:
# h = resnet50_cifar(X)

In [103]:
model(X)

tensor(3.4064, device='cuda:0', grad_fn=<MeanBackward0>)